In [1]:
%matplotlib inline
import pandas as pd
from pandas import DataFrame
import pickle
import folium
from IPython import display


In [2]:
df = pd.read_pickle('.././data/pickle/pypf_prep.pkl')
regionlookup = pd.read_csv('.././data/maps/oldnewlookupregions.csv', names=['GOR10CD', 'Area_code', 'Region'])
regionlookup['Region'] = regionlookup['Region'].str.upper()
regionlookup['Region'] = regionlookup['Region'].str.strip()
regionlookup['Region'] = regionlookup['Region'].str.replace('EAST OF ENGLAND', 'EAST')
#or_geo = '.././data/maps/gor.json'

In [3]:
maps = {}

for cause in df.Cause.unique():
    grp = df[(df['Cause'] == cause) & (df['Sex'] == 'Male')].groupby('Region')['Rate per 100,000 (standardised)'].mean()
    grp = DataFrame(grp).reset_index()
    maps[cause] = pd.merge(grp, regionlookup, on='Region')


In [4]:
maps['All Mesothelioma']

,Region,"Rate per 100,000 (standardised)",GOR10CD,Area_code
0,EAST,7.861819,E12000006,C000G
1,EAST MIDLANDS,6.482373,E12000004,C000E
2,LONDON,8.220906,E12000007,C000H
3,NORTH EAST,17.543943,E12000001,C000A
4,NORTH WEST,8.965068,E12000002,C000B
5,SOUTH EAST,9.205414,E12000008,C000J
6,SOUTH WEST,7.618154,E12000009,C000K
7,WALES,6.184247,W92000004,A0004
8,WEST MIDLANDS,6.525374,E12000005,C000F
9,YORKSHIRE AND THE HUMBER,8.646171,E12000003,C000D


In [5]:
maps['IPF'].describe()

,"Rate per 100,000 (standardised)"
count,10.000000
mean,13.155275
std,1.394424
min,11.214911
25%,12.187088
50%,12.841522
75%,14.435973
max,15.200417


In [6]:
map = folium.Map(location=[54.2, -2.45], zoom_start=5)
map.geo_json(geo_path='gor.json', data_out='GorIPF.json', data=maps['IPF'],
      columns=['GOR10CD', 'Rate per 100,000 (standardised)'],
      key_on='feature.properties.GOR10CD',
      threshold_scale=[11, 12, 13, 14, 15],
      fill_color='PuBu', fill_opacity=0.7, line_opacity=0.3,
      legend_name='Male IPF deaths by region per 100,000 population (1975-2012)')
map.create_map(path='IPF_death_per100000.html')

map = folium.Map(location=[54.2, -2.45], zoom_start=5)
map.geo_json(geo_path='gor.json', data_out='GorMES.json', data=maps['All Mesothelioma'],
      columns=['GOR10CD', 'Rate per 100,000 (standardised)'],
      key_on='feature.properties.GOR10CD',
 #%%!     threshold_scale=[6, 7, 8, 9, 10, 16],
      fill_color='PuBu', fill_opacity=0.7, line_opacity=0.3,
      legend_name='Male Mesothelioma deaths by region per 100,000 population (1975-2012)')
map.create_map(path='Mesothelioma_death_per100000.html')

map = folium.Map(location=[54.2, -2.45], zoom_start=5)
map.geo_json(geo_path='gor.json', data_out='GorASB.json', data=maps['Asbestosis'],
      columns=['GOR10CD', 'Rate per 100,000 (standardised)'],
      key_on='feature.properties.GOR10CD',
     # threshold_scale=[11, 12, 13, 14, 15],
      fill_color='PuBu', fill_opacity=0.7, line_opacity=0.3,
      legend_name='Male Asbestosis deaths by region per 100,000 population (1975-2012)')
map.create_map(path='Asbestosis_death_per100000.html')


In [7]:
display.IFrame('IPF_death_per100000.html', '100%', 500)


In [8]:
display.IFrame('Mesothelioma_death_per100000.html', '100%', 500)

In [9]:
display.IFrame('Asbestosis_death_per100000.html', '100%', 500)